<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/kaperob/system_rekomendacyjny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from datetime import datetime
import time

In [2]:
dataGrocery = pd.read_csv('GroceryStoreDataSet.csv', names=["products"])
dataGrocery.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,products
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"
5,"BREAD,TEA,BOURNVITA"
6,"MAGGI,TEA,CORNFLAKES"
7,"MAGGI,BREAD,TEA,BISCUIT"
8,"JAM,MAGGI,BREAD,TEA"
9,"BREAD,MILK"


In [3]:
dataGroceryStructured = dataGrocery['products'].apply(lambda x: x.split(','))
uniqueItems = set([item for sublist in dataGroceryStructured for item in sublist])
sorted(uniqueItems)

dataGrocery['products'] = dataGrocery['products'].str.replace('COCK', 'COKE').str.replace('SUGER', 'SUGAR')
dataGroceryStructured = dataGrocery['products'].apply(lambda x: x.split(','))

transaction_encoder = TransactionEncoder()
dataGroceryEncoded = transaction_encoder.fit(dataGroceryStructured).transform(dataGroceryStructured)
dataGroceryEncoded = pd.DataFrame(dataGroceryEncoded, columns=transaction_encoder.columns_).astype(int)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
frequent_itemsets = apriori(dataGroceryEncoded, min_support=0.15, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
max_confidence_rule = rules[rules['confidence'] == rules['confidence'].max()].head(1)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.1)
max_lift_rule = rules[rules['lift'] == rules['lift'].max()].head(1)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [5]:
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
print("Brakujące wartości w ocenach:\n", ratings.isnull().sum())

Brakujące wartości w ocenach:
 userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:

print("Brakujące wartości w filmach:\n", movies.isnull().sum())

Brakujące wartości w filmach:
 movieId    0
title      0
genres     0
dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
print("Ratings: ", ratings['rating'].nunique(), sorted(ratings['rating'].unique()))
ratings['rating'].value_counts()

Ratings:  10 [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: count, dtype: int64

In [9]:
print("Amount of ratings: ", ratings.shape[0])
print("Movies creted: ", movies['movieId'].nunique(),", Unique Titles", movies['title'].nunique())
print("Movies revied: ", movies['movieId'].nunique())
print("Amount of users: ", ratings['userId'].nunique(),", Highest Id", ratings['userId'].max())
print("Average amount of ratings per user: ", round(ratings.shape[0]/ratings['userId'].nunique(),2))
print("Average amount of ratings per movie: ", round(ratings.shape[0]/ratings['movieId'].nunique(),2))

Amount of ratings:  100836
Movies creted:  9742 , Unique Titles 9737
Movies revied:  9742
Amount of users:  610 , Highest Id 610
Average amount of ratings per user:  165.3
Average amount of ratings per movie:  10.37


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
titles = list(movies['title'])
duplicates = set([x for x in titles if titles.count(x) > 1])

print("Duplicated titles:")
for duplicate in duplicates:
  print("  -",duplicate)

movies[movies['title'].isin(duplicates)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Duplicated titles:
  - Eros (2004)
  - Confessions of a Dangerous Mind (2002)
  - War of the Worlds (2005)
  - Emma (1996)
  - Saturn 3 (1980)


,movieId,title,genres
650,838,Emma (1996),Comedy|Drama|Romance
2141,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Thriller
5601,26958,Emma (1996),Romance
5854,32600,Eros (2004),Drama
5931,34048,War of the Worlds (2005),Action|Adventure|Sci-Fi|Thriller
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [11]:
movie_titles_dict = dict(zip(movies['movieId'], movies['title']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
av = ratings.shape[0]/ratings['movieId'].nunique()
avargeOfAll = ratings['rating'].sum()/ratings.shape[0]
print("Average rating: ",av, ", Average rating of all movies: ",avargeOfAll)

Average rating:  10.369806663924312 , Average rating of all movies:  3.501556983616962


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
def bayesian_avg(ratings):
    bayesian_avg = (av * avargeOfAll + ratings.sum()) / (av + ratings.count())
    return bayesian_avg

bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
movie_stats = pd.merge(movies, bayesian_avg_ratings, on='movieId')

print(movie_stats.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      3.901634  
1                   Adventure|Children|Fantasy      3.437826  
2                               Comedy|Romance      3.299841  
3                         Comedy|Drama|Romance      3.040360  
4                                       Comedy      3.146557  


In [14]:
def create_sparse(df):
  userId = df['userId'].nunique()
  movieId = df['movieId'].nunique()
  user_mapper = dict(zip(np.unique(df["userId"]), list(range(userId))))
  movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(movieId))))
  user_inv_mapper = dict(zip(list(range(userId)), np.unique(df["userId"])))
  movie_inv_mapper = dict(zip(list(range(movieId)), np.unique(df["movieId"])))
  user_index = [user_mapper[i] for i in df['userId']]
  item_index = [movie_mapper[i] for i in df['movieId']]
  X = csr_matrix((df["rating"], (user_index, item_index)), shape=(userId,movieId))
  return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_sparse(ratings)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    X = X.T
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
n_total = X.shape[0]*X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings/n_total
sparsity_p = round(sparsity*100,2)
print(f"Matrix sparsity: {sparsity_p}%")
n_ratings_per_user = X.getnnz(axis=1)

Matrix sparsity: 1.7%


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
toyStoryID = movies[movies['title'].str.contains('Toy Story \(')]['movieId'][0]
similarMovies = find_similar_movies(toyStoryID, X, movie_mapper, movie_inv_mapper, k=10, metric='cosine')
print(f"Most similar movies (Id):  {toyStoryID}: {similarMovies}")
print(f"\nMost similar movies (Title):  {list(movies[movies['movieId']==toyStoryID]['title'])}:")
movies[movies['movieId'].isin(similarMovies)]['title']

Most similar movies (Id):  1: [3114, 480, 780, 260, 356, 364, 1210, 648, 1265]

Most similar movies (Title):  ['Toy Story (1995)']:


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:1: DeprecationWarning: invalid escape sequence '\('
<>:1: DeprecationWarning: invalid escape sequence '\('
<ipython-input-17-96d8fb164888>:1: DeprecationWarning: invalid escape sequence '\('
  toyStoryID = movies[movies['title'].str.contains('Toy Story \(')]['movieId'][0]


224             Star Wars: Episode IV - A New Hope (1977)
314                                   Forrest Gump (1994)
322                                 Lion King, The (1994)
418                                  Jurassic Park (1993)
546                            Mission: Impossible (1996)
615                  Independence Day (a.k.a. ID4) (1996)
911     Star Wars: Episode VI - Return of the Jedi (1983)
964                                  Groundhog Day (1993)
2355                                   Toy Story 2 (1999)
Name: title, dtype: object

In [18]:
toyStoryID = movies[movies['title'].str.contains('Toy Story \(')]['movieId'][0]
toyStoryID

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:1: DeprecationWarning: invalid escape sequence '\('
<>:1: DeprecationWarning: invalid escape sequence '\('
<ipython-input-18-0e81946b7eac>:1: DeprecationWarning: invalid escape sequence '\('
  toyStoryID = movies[movies['title'].str.contains('Toy Story \(')]['movieId'][0]


1

In [19]:
movies['genres'] = movies['genres'].str.lower()
movies['year'] = movies['title'].str.extract(r'(\d4)', expand=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
genreLists = list(movies['genres'].apply(lambda x: x.lower().split('|')))
genreList = []

for genres in genreLists:
  genreLists.extend(genres)

print("Set of unique genres (lowercase):", set(genreList))
genre_df = pd.DataFrame(genreList)
genre_df.value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyboardInterrupt: 

In [ ]:
children_movies_count = movies("Number of children movies:", children_movies_count)

ratings['rating_year'] = pd.to_datetime(ratings['timestamp'], unit='s').dt.year


In [ ]:
import matplotlib.pyplot as plt
ratingsJoin = ratings.merge(movies, on='movieId', how='inner')
ratings_1995 = ratingsJoin[ratingsJoin['year'] == '1995']
ratings_distribution_1995 = ratings_1995['rating'].value_counts().sort_index()
print("Distribution of 1995 film ratings:")
print(ratings_distribution_1995)
plt.figure(figsize=(10, 6))
ratings_distribution_1995.plot(kind='bar')
plt.title('Distribution of 1995 film ratings')
plt.xlabel('Rate')
plt.ylabel('Number of appearances')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


In [ ]:
actionMovies = ratingsJoin[ratingsJoin['genres'].str.contains('action', case=False)]

average_rating_action = actionMovies['rating'].mean()
print(f"Average ratings for action movies: {average_rating_action}")

top_three = actionMovies.groupby('title')['rating'].mean().sort_values(ascending=False).head(3)
print(f"\nThe three highest rated action films: {top_three}")

In [ ]:
!pip install surprise

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [21]:
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms import SVD, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

min_rating = ratings.rating.min()
max_rating = ratings.rating.max()
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[['userId','movieId' , 'rating']], reader)
trainset, testset = train_test_split(data, test_size = 0.25)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ModuleNotFoundError: No module named 'surprise'

In [ ]:
#svd
start_time_svd_train = time.time()
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)
end_time_svd_train = time.time()


In [ ]:
param_grid = {
    'n_factors': [1,2,5,10, 50]
}

start_time_svd = time.time()
gsSVD = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gsSVD.fit(data)
end_time_svd = time.time()
gsSVD.best_params

cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


In [ ]:
#knn
lgo = KNNBasic()
start_time_knn_train = time.time()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)
end_time_knn_train = time.time()

In [ ]:
param_grid = {
    'k': [1, 2, 5, 10, 20, 30, 40, 50]
}
start_time_knn = time.time()

gsKNN = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gsKNN.fit(data)
end_time_knn = time.time()
gsKNN.best_params

cross_validate(KNNBasic(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
algo = KNNBaseline()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

In [ ]:
bestKNN = gs_knn.best_estimator['rmse']
bestSVD = gs_svd.best_estimator['rmse']

start_time_knn_cross = time.time()
resultsKNN = cross_validate(bestKNN, data, measures=['rmse'], cv=5, verbose=True)
end_time_knn_cross = time.time()

start_time_svd_cross = time.time()
resultsSVD = cross_validate(bestSVD, data, measures=['rmse'], cv=5, verbose=True)
end_time_svd_cross = time.time()

print(f"\nBest KNN RMSE: {gs_knn.best_score['rmse']}")
print(f"Best SVD RMSE: {gs_svd.best_score['rmse']}")
print(f"\nBest KNN Parameters: {gs_knn.best_params['rmse']}")
print(f"Best SVD Parameters: {gs_svd.best_params['rmse']}")
print(f"\nHyperparameters KNN Training Time: {end_time_knn - start_time_knn} seconds")
print(f"Hyperparameters SVD Training Time: {end_time_svd - start_time_svd} seconds")
print(f"\nCros KNN Training Time: {end_time_knn - start_time_knn} seconds")
print(f"Cros SVD Training Time: {end_time_svd_cross - start_time_svd_cross} seconds")
print(f"\nTrain SVD Training Time: {end_time_svd_train - start_time_svd_train} seconds")
print(f"Train KNN Training Time: {end_time_knn_train - start_time_knn_train} seconds")
print(f"\nKNN Cross-validation Results: {resultsKNN}")
print(f"SVD Cross-validation Results: {resultsSVD}")